## Gather the data

In [1]:
import numpy as np
import pickle
from simtk.openmm import unit
from openmmtools.constants import kB
import pandas as pd

import matplotlib.pyplot as plt
KT_KCALMOL = kB * 300 * unit.kelvin / unit.kilocalories_per_mole

In [2]:
def compute_ddg(apo_dg, complex_dg):
    return complex_dg * KT_KCALMOL - apo_dg * KT_KCALMOL

def compute_error(apo_ddg, complex_ddg):
    return (apo_ddg**2 + complex_ddg**2)**0.5 * KT_KCALMOL

In [3]:
# Retrieve free energies

main_dir = 47
replicate = 1
length = 10
sub_dirs = list(range(0, 4)) + list(range(5, 12)) + list(range(13, 17)) + list(range(18, 28))

d_results = {}
for sub_dir in sub_dirs:
    d_results[sub_dir] = {}
    for phase in ['apo', 'complex']:
        with open(f"/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/{main_dir}/{sub_dir}/replicate_{replicate}/{phase}_free_energy_bootstrapped_{length}ns.pickle", "rb") as f:
            results = pickle.load(f)
            d_results[sub_dir][phase] = results

main_dir = 45
d_mapping = {4: 11, 12: 10, 17: 9}
for sub_dir in d_mapping.keys():
    d_results[sub_dir] = {}
    if sub_dir == 4:
        replicate = 0
    else:
        replicate = 1
    for phase in ['apo', 'complex']:
        with open(f"/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/{main_dir}/{d_mapping[sub_dir]}/replicate_{replicate}/{phase}_free_energy_bootstrapped_{length}ns.pickle", "rb") as f:
            results = pickle.load(f)
            d_results[sub_dir][phase] = results


In [4]:
# Format the data appropriately for cinnabar

titles = ["Y29F", "Y29A",  "W38F", "T42A", "W44F", "E76A", "E80A", "D35A", "D39A", "K27A", "R59A", "R83Q", "R87A", "H102A", "F29Y", "A29Y",  "F38W", "A42T", "F44W", "A76E", "A80E", "A35D", "A39D", "A27K", "A59R", "Q83R", "A87R", "A102H"]

to_plot = []

for sub_dir in range(28):
    ddg = compute_ddg(d_results[sub_dir]['apo']["Deltaf"][0, -1], d_results[sub_dir]['complex']["Deltaf"][0, -1])
    dddg = compute_error(d_results[sub_dir]['apo']["dDeltaf"][0, -1], d_results[sub_dir]['complex']["dDeltaf"][0, -1])
    if sub_dir in range(14):
        to_plot.append([titles[sub_dir], ddg, dddg*2, 0])
    else:
        to_plot.append(["-(" + titles[sub_dir] + ")", ddg * -1, dddg*2, 0])

        

In [5]:
to_plot

[['Y29F', 1.0614953535200264, 0.6698943974193585, 0],
 ['Y29A', 2.631145552280639, 0.760757261570947, 0],
 ['W38F', 0.40870070962636706, 0.6777088414962292, 0],
 ['T42A', 0.7808024898967787, 0.27478094367236605, 0],
 ['W44F', 0.10422351806408708, 0.8519971351004315, 0],
 ['E76A', 0.9600364875272691, 1.481207375275411, 0],
 ['E80A', 0.22575747080119513, 1.5491932600851857, 0],
 ['D35A', 1.6588340811494193, 1.0890173463806165, 0],
 ['D39A', 7.394021096666294, 1.2386649745825664, 0],
 ['K27A', 3.3190059071510234, 1.5164150835468635, 0],
 ['R59A', 3.3153633167397913, 1.7365219058753258, 0],
 ['R83Q', 4.988013317466471, 1.0903012756803006, 0],
 ['R87A', 11.40379760020781, 1.4650235895485961, 0],
 ['H102A', 8.754601826420796, 0.41313309617327887, 0],
 ['-(F29Y)', 0.7862032255668367, 0.8093590042665485, 0],
 ['-(A29Y)', 1.0571431914151113, 0.9761332151093736, 0],
 ['-(F38W)', 1.3573822869184209, 0.7518022979457832, 0],
 ['-(A42T)', -1.014744472610861, 0.3228863692416322, 0],
 ['-(F44W)', -0.0

In [6]:
# Assemble experimental values
experiment = [-0.1, 3.4, 1.6, 1.8, 0, 1.4, 0.5, 4.5, 7.7, 5.4, 5.2, 5.4, 5.5, 6.1, -0.1, 3.4, 1.6, 1.8, 0, 1.4, 0.5, 4.5, 7.7, 5.4, 5.2, 5.4, 5.5, 6.1]


## Plot

In [7]:
import numpy as np

import matplotlib.pylab as plt


<Figure size 432x288 with 0 Axes>

In [9]:
# Set figure size
fig, axes = plt.subplots(nrows=1, ncols=1, sharex='col', figsize=(16, 6))

# Set font size
plt.rc('font', size=15) 

# Plot
mutations = [entry[0] for entry in to_plot]
to_plot_free_energies = [entry[1] for entry in to_plot]
to_plot_uncertainties = np.array([entry[2] for entry in to_plot])
plt.scatter(mutations, to_plot_free_energies, s=50, label='calculated')
plt.errorbar(mutations, to_plot_free_energies, yerr=to_plot_uncertainties, capsize=2, fmt='none')
plt.errorbar(mutations, to_plot_free_energies, yerr=to_plot_uncertainties + 1, alpha=0.4, capsize=2, fmt='none')
plt.scatter(mutations, experiment, s=50, marker='x', color='gray', label='experimental')
plt.errorbar(mutations, experiment, yerr=2*np.array([0.1]*28), color='gray', capsize=2, fmt='none')

# Set title
DDG = r"$\Delta\Delta G$"
plt.title(f"Calculated and experimental {DDG}s per mutation (10 ns/replica AREX)")

# Show legend
plt.legend()

# Set axis labels
plt.xlabel("Mutation")
plt.ylabel(r"$\Delta\Delta G$ (kcal/mol)")

# Set x ticks
plt.xticks(rotation=45, ha='right')

# Set y ticks
plt.yticks(np.arange(-7, 15.5, 2.0))

# Ensure x-axis labels are not cut off
plt.tight_layout()

# Save plot as pdf
main_dir = 47
save_path = f"/data/chodera/zhangi/perses_benchmark/repex/perses-bnbs-paper-fourth-attempt/{main_dir}/calculated_vs_experiment_10ns.pdf"
plt.savefig(save_path, dpi=500)
plt.show()
plt.clf()